# Datos Climáticos Estado/Municipio NASA API

### Librerias

In [15]:
import os
import requests
import csv
from datetime import datetime
import pandas as pd

### Directorios

In [35]:
subdir = "./data/"

input_coord_estado = 'data/Data_In/Coordenadas_Centroide_Estado.csv' # Dir de archivo de entrada
input_coord_municipio = 'data/Data_In/Coordenadas_Centroide_Municipio.csv' # Dir de archivo de entrada

output_estado_raw = "data/Estados/Estados_Raw/" # Dir de carpeta de salida estado raw
output_estado_mod = 'data/Estados/Estados_Mod/' # Dir de carpeta de salida estado mod

output_municipio_raw = "data/Municipios/Municipios_Raw/" # Dir de carpeta de salida municipio raw
output_municipio_mod = 'data/Municipios/Municipios_Mod' # Dir de carpeta de salida

output_estado = 'data/Estados'
output_municipio = 'data/Municipios'

#os.makedirs(output_estado_raw, exist_ok=True) 
if not os.path.exists(output_estado_raw):
    os.makedirs(output_estado_raw)

# Crea la carpeta de salida si no existe
if not os.path.exists(output_estado_mod):
    os.makedirs(output_estado_mod)

# Crea la carpeta de salida si no existe
if not os.path.exists(output_municipio_raw):
    os.makedirs(output_municipio_raw)

#os.makedirs(output_municipio_raw, exist_ok=True) 
if not os.path.exists(output_municipio_mod):
    os.makedirs(output_municipio_mod)


In [30]:
coord_estado = pd.read_csv(input_coord_estado)
estado_nom = coord_estado['Estado'].tolist()
estado_ids = coord_estado['Estado_ID'].tolist()
locations = [(lat, lon) for lat, lon in zip(coord_estado['centroid_lat'], coord_estado['centroid_lon'])]

### Registro

In [ ]:
with open(subdir + "clima_info.txt", 'w') as f:
        f.write("Estadística de Producción Agrícola SIAP\n")
        info = """

        Fuente: Sistema de Estadísticas Agrícolas de México SIAP

        Descripción:
        Este registro proporciona información exhaustiva sobre las estadísticas agrícolas en México. Incluye datos cruciales, 
        como la superficie sembrada, cosechada y siniestrada, el precio medio rural, el volumen de producción y el valor de 
        la producción de cultivos tanto cíclicos como perennes.

        Cobertura Geográfica:
        Los informes abarcan las 32 entidades federativas de México, y los datos se presentan a nivel nacional, estatal, distrital y municipal.

        Rango de Tiempo:
        Los datos abarcan desde 1980 hasta 2022.

        Este registro se encuentra disponible para su consulta y análisis, proporcionando una visión completa de la agricultura 
        en México a lo largo de las décadas, y ofreciendo información crucial para la toma de decisiones y la planificación en el sector agrícola.

        """ 
        f.write(info + '\n')
        f.write("Descargado el " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
        f.write('\n')
        f.write("Desde: " + url + "\n")

### Estados

In [21]:
parameter = 'TS,TS_MAX,TS_MIN,T2M,T2M_MAX,T2M_MIN,T2MDEW,T2MWET,PS,WS2M,RH2M,EVPTRNS,GWETPROF,CLRSKY_DAYS,PRECTOTCORR,SG_DAY_HOURS,MIDDAY_INSOL'

start_day = '20200101'
end_day =  datetime.now().strftime('%Y%m%d') # Los Ultimos dias del mes actual vienen con faltantes


base_url = r"https://power.larc.nasa.gov/api/temporal/daily/point?parameters={parameter}&community=RE&longitude={longitude}&latitude={latitude}&start={start_day}&end={end_day}&format=CSV"

In [22]:
for estado_nom, latitude, longitude in zip(coord_estado['Estado'], coord_estado['centroid_lat'], coord_estado['centroid_lon']):
    api_request_url = base_url.format(longitude=longitude, latitude=latitude, parameter=parameter, start_day=start_day, end_day=end_day)

    response = requests.get(url=api_request_url, verify=True, timeout=30.00)

    if response.status_code == 200:
        content = response.content.decode('utf-8')
        filename = f"Data_{estado_nom}_{start_day}-{end_day}.csv"
        filepath = os.path.join(output_estado_raw, filename)

        with open(filepath, 'wb') as file_object:
            file_object.write(response.content)

### Dataframe de prueba

In [23]:
archivos_csv = [archivo for archivo in os.listdir(output_estado_raw) if archivo.endswith('.csv')]

if archivos_csv:
    # Lee el primer archivo CSV de la lista
    primer_archivo = archivos_csv[0]
    ruta_completa_primer_csv = os.path.join(output_estado_raw, primer_archivo)
    df_test = pd.read_csv(ruta_completa_primer_csv,skiprows=25)

print(df_test.shape)
df_test.head()

(1403, 20)


,YEAR,MO,DY,TS,TS_MAX,TS_MIN,T2M,T2M_MAX,T2M_MIN,T2MDEW,T2MWET,PS,WS2M,RH2M,EVPTRNS,GWETPROF,CLRSKY_DAYS,PRECTOTCORR,SG_DAY_HOURS,MIDDAY_INSOL
0,2020,1,1,13.37,22.76,6.50,13.30,20.08,7.35,7.12,10.21,79.30,3.91,68.44,0.04,0.43,0.0,6.93,10.0,12.09
1,2020,1,2,10.40,18.46,1.71,10.26,16.07,3.16,5.71,7.99,79.07,4.16,75.94,0.03,0.43,0.0,6.69,10.0,14.65
2,2020,1,3,7.87,21.67,-1.38,7.77,18.33,0.31,-1.83,2.97,79.56,2.52,59.25,0.06,0.44,1.0,0.01,10.0,19.79
3,2020,1,4,6.33,20.80,-3.20,6.06,17.44,-2.65,-8.14,-1.04,80.13,3.05,42.31,0.06,0.43,0.0,0.00,10.0,16.03
4,2020,1,5,7.31,23.21,-3.40,7.55,20.03,-1.74,-8.03,-0.24,80.07,2.41,39.00,0.07,0.43,1.0,0.00,10.0,19.88


### Transformación de datos

In [24]:
# Obtén la lista de archivos CSV en la carpeta
archivos_csv = [archivo for archivo in os.listdir(output_estado_raw) if archivo.endswith('.csv')]
n = 0 # Contador de toda vida 

# Procesa y transforma cada archivo CSV
for archivo_csv in archivos_csv:

    # Lee el archivo CSV, salta las primeras 25 filas y agrega una columna con el nombre del estado
    ruta_completa = os.path.join(output_estado_raw, archivo_csv)
    df = pd.read_csv(ruta_completa, skiprows=25)
    estado_nom = archivo_csv.split('_')[1]
    df.insert(0, 'Estado_Nom', estado_nom)

    # Agrega una columna con el ID del estado
    df.insert(1, 'Estado_ID', estado_ids[n])
    n+=1
 
    # Transforma las columnas YEAR, MO y DY en una sola columna de tipo fecha
    df = df.rename(columns={'YEAR': 'year', 'MO': 'month', 'DY': 'day'})
    df.insert(0, 'Fecha', pd.to_datetime(df[['year', 'month', 'day']], format='%Y%m%d'))
    df.drop(columns=['year', 'month', 'day'], inplace=True) 

    # Guarda el archivo CSV procesado en la carpeta de salida
    ruta_completa_salida = os.path.join(output_estado_mod, archivo_csv)
    with open(ruta_completa_salida, 'w') as file_object:
        df.to_csv(file_object, index=False,lineterminator='\n')

### Concatecion de Dataframes

In [25]:
# Obtener la lista de archivos CSV en la carpeta
archivos_csv = [archivo for archivo in os.listdir(output_estado_mod)]

# Lista para almacenar los DataFrames individuales
dataframes = []

# Lee y combina los archivos CSV en un solo DataFrame
for archivo_csv in archivos_csv:
    ruta_completa = os.path.join(output_estado_mod, archivo_csv)
    df = pd.read_csv(ruta_completa)
    dataframes.append(df)

# Combina los DataFrames en uno solo
df_combinado = pd.concat(dataframes, ignore_index=True)

output_filename = 'Data_Climatic_Estados_Concat.csv'  # Nombre del archivo de salida
output_path = os.path.join(output_estado, output_filename)

# Guarda el DataFrame combinado en el archivo CSV utilizando with open
with open(output_path, 'w', newline='') as file_object:
    df_combinado.to_csv(file_object, index=False)

df_combinado

,Fecha,Estado_Nom,Estado_ID,TS,TS_MAX,TS_MIN,T2M,T2M_MAX,T2M_MIN,T2MDEW,T2MWET,PS,WS2M,RH2M,EVPTRNS,GWETPROF,CLRSKY_DAYS,PRECTOTCORR,SG_DAY_HOURS,MIDDAY_INSOL
0,2020-01-01,Aguascalientes,MX-ZAC,13.37,22.76,6.50,13.30,20.08,7.35,7.12,10.21,79.30,3.91,68.44,0.04,0.43,0.0,6.93,10.0,12.09
1,2020-01-02,Aguascalientes,MX-ZAC,10.40,18.46,1.71,10.26,16.07,3.16,5.71,7.99,79.07,4.16,75.94,0.03,0.43,0.0,6.69,10.0,14.65
2,2020-01-03,Aguascalientes,MX-ZAC,7.87,21.67,-1.38,7.77,18.33,0.31,-1.83,2.97,79.56,2.52,59.25,0.06,0.44,1.0,0.01,10.0,19.79
3,2020-01-04,Aguascalientes,MX-ZAC,6.33,20.80,-3.20,6.06,17.44,-2.65,-8.14,-1.04,80.13,3.05,42.31,0.06,0.43,0.0,0.00,10.0,16.03
4,2020-01-05,Aguascalientes,MX-ZAC,7.31,23.21,-3.40,7.55,20.03,-1.74,-8.03,-0.24,80.07,2.41,39.00,0.07,0.43,1.0,0.00,10.0,19.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44891,2023-10-30,Zacatecas,MX-AGU,19.26,30.80,11.84,17.87,25.23,11.58,11.33,14.61,79.88,2.61,69.69,-999.00,-999.00,-999.0,1.16,-999.0,-999.00
44892,2023-10-31,Zacatecas,MX-AGU,13.39,24.35,6.09,11.45,18.00,6.09,6.73,9.09,80.17,3.59,74.50,-999.00,-999.00,-999.0,0.35,-999.0,-999.00
44893,2023-11-01,Zacatecas,MX-AGU,14.76,29.64,5.92,12.65,21.80,5.69,6.27,9.46,80.23,2.41,68.94,-999.00,-999.00,-999.0,0.23,-999.0,-999.00
44894,2023-11-02,Zacatecas,MX-AGU,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.0,-999.00,-999.0,-999.00


### Municipios

In [31]:
coord_municipio = pd.read_csv(input_coord_municipio)
municipio_nom = coord_municipio['Municipio'].tolist()
municipio_ids = coord_municipio['Municipio_CVE'].tolist()
locations_mun = [(lat, lon) for lat, lon in zip(coord_municipio['centroid_lat'], coord_municipio['centroid_lon'])]
coord_municipio.head()

,Estado_CVE,Municipio_CVE,Municipio,centroid_lat,centroid_lon,Pertenece
0,1,2,Asientos,22.126737,-102.045605,True
1,1,11,San Francisco de los Romo,22.033110,-102.228397,True
2,1,7,Rincón de Romos,22.267232,-102.340802,True
3,1,8,San José de Gracia,22.146885,-102.525446,True
4,1,5,Jesús María,21.932282,-102.445932,True


In [32]:
# Mismos parametros que para los estados

start_day = '20200101' # mismo que para los estados
end_day =  datetime.now().strftime('%Y%m%d') # Los Ultimos dias del mes actual vienen con faltantes

# mismo base_url que para los estados

### Descarga de la API

In [ ]:
for estado_cve, municipio_cve, municipio_nom, latitude, longitude in zip(coord_municipio['Estado_CVE'], coord_municipio['Municipio_CVE'], coord_municipio['Municipio'], coord_municipio['centroid_lat'], coord_municipio['centroid_lon']):
    api_request_url = base_url.format(longitude=longitude, latitude=latitude, parameter=parameter, start_day=start_day, end_day=end_day)

    response = requests.get(url=api_request_url, verify=True, timeout=30.00)

    if response.status_code == 200:
        content = response.content.decode('utf-8')
        filename = f"Data-{estado_cve}-{municipio_cve}_{municipio_nom}_{start_day}-{end_day}.csv"
        filepath = os.path.join(output_municipio_raw, filename)

        with open(filepath, 'wb') as file_object:
            file_object.write(response.content)

### Prueba

In [33]:
archivos_csv = [archivo for archivo in os.listdir(output_municipio_raw) if archivo.endswith('.csv')]

if archivos_csv:
    # Lee el primer archivo CSV de la lista
    primer_archivo = archivos_csv[0]
    ruta_completa_primer_csv = os.path.join(output_municipio_raw, primer_archivo)
    df_test = pd.read_csv(ruta_completa_primer_csv,skiprows=25)

print(df_test.shape)
df_test.head()

(1402, 20)


,YEAR,MO,DY,TS,TS_MAX,TS_MIN,T2M,T2M_MAX,T2M_MIN,T2MDEW,T2MWET,PS,WS2M,RH2M,EVPTRNS,GWETPROF,CLRSKY_DAYS,PRECTOTCORR,SG_DAY_HOURS,MIDDAY_INSOL
0,2020,1,1,12.91,23.01,6.02,12.88,19.87,7.19,7.03,9.95,78.76,5.12,69.81,0.01,0.50,0.0,13.81,10.0,12.09
1,2020,1,2,11.03,19.21,3.86,10.91,16.86,5.05,6.05,8.48,78.48,6.74,74.31,0.01,0.51,0.0,5.85,10.0,14.65
2,2020,1,3,8.24,21.95,0.26,8.14,18.27,1.51,-1.43,3.36,79.02,3.66,59.56,0.02,0.51,1.0,0.02,10.0,19.79
3,2020,1,4,6.03,20.34,-3.12,5.65,16.12,-1.79,-6.48,-0.42,79.68,3.70,49.56,0.01,0.51,0.0,0.00,10.0,16.03
4,2020,1,5,6.23,22.62,-4.48,6.79,18.73,-1.85,-7.67,-0.44,79.60,3.02,42.12,0.02,0.51,1.0,0.00,10.0,19.88


### Transforma DataFrame

In [36]:
# Obtén la lista de archivos CSV en la carpeta
archivos_csv = [archivo for archivo in os.listdir(output_municipio_raw) if archivo.endswith('.csv')]

# Procesa y transforma cada archivo CSV
for archivo_csv in archivos_csv:

    # Lee el archivo CSV, salta las primeras 25 filas y agrega una columna con el nombre del estado
    ruta_completa = os.path.join(output_municipio_raw, archivo_csv)
    df = pd.read_csv(ruta_completa, skiprows=25)
    municipio_nom = archivo_csv.split('_')[1]
    df.insert(0, 'Municipio_Nom', municipio_nom)

    estado_cve = archivo_csv.split('_')[0].split('-')[1]
    df.insert(1, 'Estado_CVE', estado_cve)

    municipio_cve = archivo_csv.split('_')[0].split('-')[2]
    df.insert(2, 'Municipio_CVE', municipio_cve)
 
    # Transforma las columnas YEAR, MO y DY en una sola columna de tipo fecha
    df = df.rename(columns={'YEAR': 'year', 'MO': 'month', 'DY': 'day'})
    df.insert(0, 'Fecha', pd.to_datetime(df[['year', 'month', 'day']], format='%Y%m%d'))
    df.drop(columns=['year', 'month', 'day'], inplace=True) 

    # Guarda el archivo CSV procesado en la carpeta de salida
    ruta_completa_salida = os.path.join(output_municipio_mod, archivo_csv)
    with open(ruta_completa_salida, 'w') as file_object:
        df.to_csv(file_object, index=False,lineterminator='\n')

### Concatena DataFrames

In [37]:
# Obtener la lista de archivos CSV en la carpeta
archivos_csv = [archivo for archivo in os.listdir(output_municipio_mod)]

# Lista para almacenar los DataFrames individuales
dataframes = []

# Lee y combina los archivos CSV en un solo DataFrame
for archivo_csv in archivos_csv:
    ruta_completa = os.path.join(output_municipio_mod, archivo_csv)
    df = pd.read_csv(ruta_completa)
    dataframes.append(df)

# Combina los DataFrames en uno solo
df_combinado = pd.concat(dataframes, ignore_index=True)

output_filename = 'Data_Climatic_Municipios_Concat.csv'  # Nombre del archivo de salida
output_path = os.path.join(output_municipio, output_filename)

# Guarda el DataFrame combinado en el archivo CSV utilizando with open
with open(output_path, 'w', newline='') as file_object:
    df_combinado.to_csv(file_object, index=False)

df_combinado

,Fecha,Municipio_Nom,Estado_CVE,Municipio_CVE,TS,TS_MAX,TS_MIN,T2M,T2M_MAX,T2M_MIN,...,T2MWET,PS,WS2M,RH2M,EVPTRNS,GWETPROF,CLRSKY_DAYS,PRECTOTCORR,SG_DAY_HOURS,MIDDAY_INSOL
0,2020-01-01,El Llano,1,10,12.91,23.01,6.02,12.88,19.87,7.19,...,9.95,78.76,5.12,69.81,0.01,0.50,0.0,13.81,10.0,12.09
1,2020-01-02,El Llano,1,10,11.03,19.21,3.86,10.91,16.86,5.05,...,8.48,78.48,6.74,74.31,0.01,0.51,0.0,5.85,10.0,14.65
2,2020-01-03,El Llano,1,10,8.24,21.95,0.26,8.14,18.27,1.51,...,3.36,79.02,3.66,59.56,0.02,0.51,1.0,0.02,10.0,19.79
3,2020-01-04,El Llano,1,10,6.03,20.34,-3.12,5.65,16.12,-1.79,...,-0.42,79.68,3.70,49.56,0.01,0.51,0.0,0.00,10.0,16.03
4,2020-01-05,El Llano,1,10,6.23,22.62,-4.48,6.79,18.73,-1.85,...,-0.44,79.60,3.02,42.12,0.02,0.51,1.0,0.00,10.0,19.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3446111,2023-10-29,Milpa Alta,9,9,17.36,26.55,11.39,17.51,23.84,12.43,...,15.52,79.16,0.81,79.31,-999.00,-999.00,-999.0,1.07,-999.0,-999.00
3446112,2023-10-30,Milpa Alta,9,9,16.24,24.74,10.30,16.44,22.36,11.41,...,14.11,79.20,1.09,76.50,-999.00,-999.00,-999.0,0.08,-999.0,-999.00
3446113,2023-10-31,Milpa Alta,9,9,15.48,24.34,9.40,15.36,21.87,10.44,...,13.46,79.28,1.26,79.75,-999.00,-999.00,-999.0,0.89,-999.0,-999.00
3446114,2023-11-01,Milpa Alta,9,9,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,...,-999.00,-999.00,-999.00,-999.00,-999.00,-999.00,-999.0,-999.00,-999.0,-999.00
